In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os,sys,re
from itertools import count
from time import sleep

def clean_text(text):
    text = text.replace("\n", "")
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', text)
    return cleaned_text

def get_podcast_mp3(pod_cast_id):
    page_url = "http://www.podbbang.com/podbbangchnew/episode_list"

    headers ={
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
            (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36',
    }

    # for page_id in range(2):
    for page_id in count(1):
        params_dict ={
            'id' : pod_cast_id,
            'page' : page_id,
        }
        res = requests.get(page_url, params = params_dict, headers = headers)
        res.encoding = 'utf-8'
        # print(res.status_code, res.encoding)
        html = res.text
        soup = BeautifulSoup(html, 'html.parser')
 
        for li_tag in soup.select('li'):
            try:
                title = li_tag.find('dt')['title']
                if li_tag.find('a'):
                    link = urljoin(page_url, li_tag.find('a')['href'])
                else:
                    link = None
                print(title,link)
            except(TypeError, KeyError): # if li tag doesn't exist anymore, >> type error
                print('End!')
                return
            else:
                req_headers ={
                    'Referer' : 'http://www.podbbang.com/ch/' + str(pod_cast_id),
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 \
                        (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36',
                }
                if link:
                    mp3_bin = requests.get(url= link, headers = req_headers).content
                    title = clean_text(title)
                    file_name = f'{title}.mp3'
                    if not os.path.isdir('./mp3'):
                        os.mkdir('./mp3')
                    with open('./mp3/' + file_name, 'wb') as f:
                        f.write(mp3_bin)
                sleep(1) # sleep for a one second

In [ ]:
get_podcast_mp3(1773764)